In [1]:
import pandas as pd

gender = []
percent = []
averageprice = []
purchasecount = []
totalpurchase = []
avgtotal = []

In [2]:
# Read in the purchasing data file and place them into our dataframe purchase_df
purchase_df = pd.read_csv('purchase_data.csv')

# Print out the first 5 lines of the dataframe
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# This cell doesn't create something used later, but checks that all rows have all values in them (no empty cells)
# We will assume that the equal numbers mean that all rows are full, not that each column has an equal number of blanks
# If errors pop up later we will revisit this assumption
purchase_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
# Retrieve all of the unique names in the SN or screen name column and place them into a numpy array screennames
screennames = purchase_df['SN'].unique()

# The .nunique() counts the number of unique players (which are stored in the screennames array)
playernum = purchase_df['SN'].nunique()

# Display the desired value in text format
print("Total Number of Players: " + str(playernum))

# Place the data into a datafram to match the desired formatting
total_df = pd.DataFrame({"Total Players": [playernum]})
total_df

Total Number of Players: 576


,Total Players
0,576


In [5]:
# Calculate the number of unique items in the same way as above
itemnum = purchase_df['Item Name'].nunique()

# Calculate the average price of all items purchased using .mean()
avgprice = purchase_df['Price'].mean()

# Since we checked above that all cells are filled just choose one of them to count
purchasenum = purchase_df['Purchase ID'].count()

# Sum up the final column to get the total revenue generated
stacksgenerated = purchase_df['Price'].sum()

# Create the raw dataframe before formatting
summary_df = pd.DataFrame({'Number of Unique Items':[itemnum], 
                          'Average Price':[avgprice], 
                          'Number of Purchases':[purchasenum], 
                          'Revenue Generated':[stacksgenerated]})

# Format the money columns of the dataframe
summary_df['Average Price'] = summary_df['Average Price'].map("${:,.2f}".format)
summary_df['Revenue Generated'] = summary_df['Revenue Generated'].map("${:,.2f}".format)
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Revenue Generated
0,179,$3.05,780,"$2,379.77"


In [6]:
# Count the initial numer of each gender category in the original dataset (knowing that this contains duplicates)
purchase_df['Gender'].value_counts()

Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64

In [7]:
# Pull out only the columns that are relevant to this analysis from the larger dataframe
player_df = purchase_df[['SN','Gender']]

# Drop the duplicate screen names to leave only unique names behind
player_df = player_df.drop_duplicates('SN')

# Visualize the remaining counts and note that it is smaller than the original
gender_breakdown = player_df['Gender'].value_counts()

# Take the values in gender breakdown for our summary dataframe
for i in gender_breakdown:
    gender.append(i)

# Normalize = True normalizes the counts by the total number of values, multiplying by 100 converts to percent
player_percentage = player_df['Gender'].value_counts(normalize = True) * 100
for j in player_percentage:
    percent.append(j)
    
# Create the dataframe with the raw data
gender_df = pd.DataFrame({'Total Count': gender, 'Percentage of Players': percent})

indices = pd.Series(['Male', 'Female', 'Other / Non-Disclosed'])
gender_df = gender_df.set_index(indices)
gender_df['Percentage of Players'] = gender_df['Percentage of Players'].map("{:.2f}%".format)
gender_df


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
# Count up the number of purchases made by each gender
purchasenum = purchase_df.groupby('Gender')['SN'].count()

# Convert this series into a dataframe
gendersummary_df = purchasenum.to_frame()

# Group by gender then calculate the average price and total value of items purchased
averageprice = purchase_df.groupby('Gender')['Price'].mean()
totalpurchase = purchase_df.groupby('Gender')['Price'].sum()


# Place these series into the summary dataframe
gendersummary_df["Average Price"] = averageprice
gendersummary_df["Total Purchase"] = totalpurchase



gendersummary_df

,SN,Average Price,Total Purchase
Gender,,,
Female,113,3.203009,361.94
Male,652,3.017853,1967.64
Other / Non-Disclosed,15,3.346000,50.19


In [9]:
# Take the unique counts from the earlier dataframe and use it to calculate average total
gendersummary_df['Avg Total'] = gendersummary_df['Total Purchase'] / gender_df['Total Count']

# Display the raw dataframe with all of the values in place
gendersummary_df

,SN,Average Price,Total Purchase,Avg Total
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [10]:
#Correct the formatting of the table to use both money and comma format
gendersummary_df['Average Price'] = gendersummary_df['Average Price'].map("${:,.2f}".format)
gendersummary_df['Total Purchase'] = gendersummary_df['Total Purchase'].map("${:,.2f}".format)
gendersummary_df['Avg Total'] = gendersummary_df['Avg Total'].map("${:,.2f}".format)

#Display the dataframe with the corrected formatting
gendersummary_df


,SN,Average Price,Total Purchase,Avg Total
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [11]:
# Create a smaller dataframe with the relevant data for this analysis
age_purchase_df = purchase_df[['SN','Age']]
age_purchase_df = age_purchase_df.drop_duplicates('SN')

# Create the bins
bins = [0,9,14,19,24,29,34,39,100]

# Name the bins
bin_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Bin the data and store that in a new dataframe
age_purchase_df['Age Bin'] = pd.cut(age_purchase_df['Age'], bins, labels = bin_names)
age_demo_df = age_purchase_df['Age Bin'].value_counts().to_frame()

# Calculate the percentage column
age_demo_df['Percentage of Players'] = age_demo_df['Age Bin'] / age_purchase_df['SN'].count() * 100
age_demo_df = age_demo_df.rename(columns = {'Age Bin': 'Total Count'})

# Sort the data to order by the original bin order
age_demo_df = age_demo_df.sort_index(0)
age_demo_df

,Total Count,Percentage of Players
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-39,31,5.381944
40+,12,2.083333


In [12]:
# Reformat the percentage column to round and add the percent symbol
age_demo_df['Percentage of Players'] = age_demo_df['Percentage of Players'].map("{:.2f}%".format)
age_demo_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [13]:
# Add a column to purchase_df to identify the bin each person falls into
purchase_df['Age Bin'] = pd.cut(purchase_df['Age'], bins, labels = bin_names)
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [17]:
# Create a dataframe with the purchase counts by age group
age_summary_df = purchase_df.groupby('Age Bin')['SN'].count().to_frame()

# Populate the columns in the same way as we did for the gender table above
age_summary_df['Average Purchase Price'] = purchase_df.groupby('Age Bin')['Price'].mean()
age_summary_df['Total Purchase Value'] = purchase_df.groupby('Age Bin')['Price'].sum()

# Same idea as above taking the count from our previous de-duplicated counts
age_summary_df['Avg Total Purchase Per Person'] = age_summary_df['Total Purchase Value'] / age_demo_df['Total Count']

# Rename that initial column
age_summary_df = age_summary_df.rename(columns = {'SN': 'Purchase Count'})
age_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Bin,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [18]:
age_summary_df['Average Purchase Price'] = age_summary_df['Average Purchase Price'].map("${:,.2f}".format)
age_summary_df['Total Purchase Value'] = age_summary_df['Total Purchase Value'].map("${:,.2f}".format)
age_summary_df['Avg Total Purchase Per Person'] = age_summary_df['Avg Total Purchase Per Person'].map("${:,.2f}".format)
age_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Bin,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [36]:
# Create our topspenders dataframe using purchase number as our initial column
topspenders_df = purchase_df.groupby('SN')['Purchase ID'].count().to_frame()

# Group by and create the summary values for average and total price
topspenders_df['Average Purchase Price'] = purchase_df.groupby('SN')['Price'].mean()
topspenders_df['Total Purchase Value'] = purchase_df.groupby('SN')['Price'].sum()

topspenders_df = topspenders_df.rename(columns = {'Purchase ID': 'Purchase Count'})

# Sort the dataframe by the total purchase value
topspenders_df = topspenders_df.sort_values('Total Purchase Value', ascending = False)

topspenders_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [37]:
# As is very familiar now we format our columns into dollar format and re-visualize the top 5 rows
topspenders_df['Average Purchase Price'] = topspenders_df['Average Purchase Price'].map("${:,.2f}".format)
topspenders_df['Total Purchase Value'] = topspenders_df['Total Purchase Value'].map("${:,.2f}".format)
topspenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
